In [1]:
import datafaucet as dfc

In [2]:
# start the engine
engine = dfc.engine('spark')

created SparkEngine
Init engine "spark"
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [52]:
df = dfc.range(1000)

In [53]:
from pyspark.sql import functions as F

df = (df
    .cols.create('prefix').randchoice(['032', '033','035', '095', '093', '098'])  
    .cols.create('number').randint(1111111, 9999999)  
    .cols.get('number').cast('string')
    .cols.create('f').fake('first_name')
    .cols.create('l').fake('last_name')
).cache()

#concat trunc + code + number
df = df.select(F.concat('prefix', 'number').alias('number'), 'f', 'l')

In [54]:
df.count()

1000

In [55]:
df.data.grid(5)

In [55]:
df= df.withColumn('f', F.length('f'))

cols = df.columns - 'f'
df = df.select(F.length('f').alias('f'), *cols)

df =df.cols.get('f').apply(F.length)

,number,f,l
0,0984849568,Steven,Jones
1,0324116719,Sean,Jenkins
2,0952016637,Robert,Strong
3,0327252245,Katherine,Stevenson
4,0321199561,Jonathan,Cruz


In [57]:
from pyspark.sql import functions as F
from datafaucet.spark.functions import obscure

def mobile_obscure(key):
        return lambda c: F.concat(F.substring(c, 2, 3), F.lit('%'), obscure(key)(c))
    
res = df.cols.get('number').apply(mobile_obscure('mysecret'))
res.data.grid(5)

,number,f,l
0,984%i3XwDox2i3GMdgQA,Steven,Jones
1,324%i/VyDAxyDnaOcQAA,Sean,Jenkins
2,952%i3VwCw92DnaK8wMA,Robert,Strong
3,327%i/VyDAp0D3eL9AEA,Katherine,Stevenson
4,321%i/VyDAnyjnGM9gAA,Jonathan,Cruz


In [58]:
from pyspark.sql import functions as F
from datafaucet.spark.functions import unravel

def mobile_unravel(key):
        return lambda c: unravel(key)(F.element_at(F.split(c, '%'),2))

res.cols.get('number').apply(mobile_unravel('mysecret')).data.grid(5)

,number,f,l
0,0984849568,Steven,Jones
1,0324116719,Sean,Jenkins
2,0952016637,Robert,Strong
3,0327252245,Katherine,Stevenson
4,0321199561,Jonathan,Cruz


In [12]:
%%time
df.cols.get('y').encrypt(key).cols.get('y').data.grid().iloc[1,0]

CPU times: user 16.8 ms, sys: 525 µs, total: 17.3 ms
Wall time: 1.18 s


'Z0FBQUFBQmQwNDRuMHVMVjNEMlN0QzdyQmlLNEo2cVVYWWg0czNZYnZ6b1pPSDh6Nk5veFBPeHpCcElJZVZPeE9VNVpjOW5KWXJMU1Z6amFLT3plbFFjVHlyYVBNTzRnTEE9PQ=='

In [25]:
df = (df
     .cols.get('f').obscure('mysecret')
     .cols.get('y').encrypt(key)
)

In [30]:
df.cols.get('f', 'y').data.grid(5)

,f,y
0,sxKQ5+QS5gAA,Z0FBQUFBQmQwdnJKNVdoMlFHbFBQU05ER0NObmJTVVhsR2...
1,U5BglAEA,Z0FBQUFBQmQwdnJKdHB6WG10MURFWEpIWkVuTmhablA1S2...
2,sxQQZWaSkQEA,Z0FBQUFBQmQwdnJKQW52M0x2ejBTaDhNcHQ3WlhUNUNnRj...
3,0+cWYuQXlwEA,Z0FBQUFBQmQwdnJKTTNXM3Q4c2VEZkVteEo3YVF1LVpCdn...
4,U5Nh5eEFAA==,Z0FBQUFBQmQwdnJKd3prX3MyNG1QdXdrN3Z6N2oxYUljdF...


In [31]:
df = (df
     .cols.get('f').unravel('mysecret')
     .cols.get('y').decrypt(key)
)

In [32]:
df.cols.get('f', 'y').data.grid(5)

,f,y
0,William,David Berger
1,Mary,Amy Owen
2,Tiffany,Sarah Conley
3,Bradley,Kyle Beltran
4,Kevin,Michelle Ayers
